<a href="https://colab.research.google.com/github/jogianni/sectionBapp/blob/main/SectionBWhatsapp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**IMPORTS**

In [1]:
import pandas as pd
import re
import io
import os
from google.colab import drive
from google.colab import files
import numpy as np
import datetime
!pip install emoji --upgrade
import emoji
from collections import Counter

     |████████████████████████████████| 170 kB 6.5 MB/s 
  Created wheel for emoji: filename=emoji-1.6.1-py3-none-any.whl size=169314 sha256=82055215d2878e12b06d8410644525452ae8495663293b5fb1e2f6b176f8b5ee
  Stored in directory: /root/.cache/pip/wheels/ea/5f/d3/03d313ddb3c2a1a427bb4690f1621eea60fe6f2a30cc95940f
Successfully built emoji


In [2]:
drive.mount('/content/gdrive')
root_path = 'gdrive/My Drive/IESE/'

Mounted at /content/gdrive


In [3]:
#os.listdir('/content/gdrive/My Drive/IESE')
os.chdir('/content/gdrive/My Drive/IESE')

In [25]:
def read_file(file):
    '''Reads Whatsapp text file into a list of strings'''
    x = open(file,'r', encoding = 'utf-8') #Opens the text file into variable x but the variable cannot be explored yet
    y = x.read() #By now it becomes a huge chunk of string that we need to separate line by line
    content = y.splitlines() #The splitline method converts the chunk of string into a list of strings
    return content

chat = read_file('sectionB10.24.2021.txt')
#051920212300_cut
#sectionB10.24.2021

In [26]:
#Drop lines that don't start with '[' - the class tends to send lots of lists with carriage returns. We don't want to count thoe as individual messages and it's hard to identify their owner, so we drop them
chat = [x for x in chat if x.startswith('[')]

In [27]:
#reads date in time if your phone is set to miliatary time
#if you phone is am/pm d{2}.{3} is needed to replace the last d{2} in time 
regex = re.compile(r'\[(?P<date>\d{1,2}\/\d{1,2}\/\d{2})\s(?P<time>\d{1,2}:\d{2}:\d{2})]\s(?P<Name>[^:]*):\s(?P<content>.+|\n+(?!)\[\d{2}\/\d{2}\/\d{4})')

print (len(chat))
chat_matches = [regex.search(content) for content in chat]
chat_list = [m.groupdict() for m in chat_matches if not m is None]
print (len(chat_list))
#print (chat_list)

24215
24062


In [ ]:
#reads date in time if your phone is set to am pm time

#regex = re.compile(r'\[(?P<date>\d{1,2}\/\d{1,2}\/\d{2})\s(?P<time>\d{1,2}:\d{2}:\d{2}.{3})]\s(?P<Name>[^:]*):\s(?P<content>.+|\n+(?!)\[\d{2}\/\d{2}\/\d{4})')

#print (len(chat))
#chat_matches = [regex.search(content) for content in chat]
#chat_list = [m.groupdict() for m in chat_matches if not m is None]
#print (len(chat_list))
#print (chat_list)

In [28]:
#make Dataframe
df = pd.DataFrame(chat_list)

df['datetime'] = pd.to_datetime(df['date'] + ' ' + df['time'])

df['date'] = [datetime.datetime.strptime(x, '%d/%m/%y') for x in df['date']]

#add some datums
df['msg_len']  = df['content'].str.len()
#Get word and message count
df['Letter_Count'] = df['content'].apply(lambda s : len(s.replace(' ','')))
df['Word_Count'] = df['content'].apply(lambda s : len(s.split(' ')))
#letter and word totals
df['Letter_Count'].sum(), df['Word_Count'].sum()

(739905, 153996)

In [29]:
#read in coa
coa_df = pd.read_csv('COA.csv') 

In [30]:
merged_df = pd.merge(left=df, right=coa_df, how='left', left_on='Name', right_on='Name')

In [ ]:
merged_df.head()

,date,time,Name,content,datetime,msg_len,Letter_Count,Word_Count,Team,Country of Origin,Diagnosed Country
0,2019-09-02,12:37:01 PM,🏳️‍🌈 Best Section IESE 2021!,‎Los mensajes y las llamadas están cifrados de...,2019-02-09 12:37:01,143,121,23,NaN,NaN,NaN
1,2019-09-02,12:42:26 PM,‪+1 (514) 999‑4123‬,Lol love the logo! Well done Juli 🙌🙌,2019-02-09 12:42:26,36,29,8,NaN,NaN,NaN
2,2019-09-02,12:42:32 PM,Lucas Afonso,🙌🏻🙌🏻🙌🏻,2019-02-09 12:42:32,6,6,1,2.0,NaN,Brazil
3,2019-09-02,12:43:40 PM,Barbara Handam IESE,👏🏻👏🏻👏🏻,2019-02-09 12:43:40,6,6,1,4.0,Brazil,Brazil
4,2019-09-02,12:43:41 PM,Chris Coraggio IESE,B = Bad Bitches,2019-02-09 12:43:41,15,12,4,8.0,United States,United States


In [ ]:
merged_df.to_csv("051920212300_converted.csv",index=False)

In [ ]:
def extract_emojis(str):
    return ''.join(c for c in str if c in emoji.UNICODE_EMOJI)


In [ ]:
def Count_Emojis(df, only_one_per_message = False):
    series = df['content']
    all_words = ""
    for sentence in series:
        all_words += extract_emojis(sentence)
    word_count = Counter(all_words)
    
    
    ordered = {}
    
    for key, number in word_count.most_common()[:50]:
        ordered[key] = []
    
    for sentence in series:
        sentence_count = Counter(extract_emojis(sentence))
        
        for word in ordered:
            if only_one_per_message:
                count = 1 if sentence_count[word] else 0
            else:
                count = sentence_count[word] if sentence_count[word] else 0
            ordered[word] += [count] 
             
            
    ordered['content'] = list(series)
    ordered['date'] = list(df['date'])
    ordered['Name'] = list(df['Name'])
    ordered['Country of Origin'] = list(df['Country of Origin'])
    ordered['Diagnosed Country'] = list(df['Diagnosed Country'])
    ordered['Team'] = list(df['Team'])
    
    return pd.DataFrame(ordered)

In [ ]:
emojis_counts = Count_Emojis(merged_df)

In [ ]:
emojis_counts.head(20)

,content,date,Name,Country of Origin,Diagnosed Country,Team
0,‎Los mensajes y las llamadas están cifrados de...,2019-09-02,🏳️‍🌈 Best Section IESE 2021!,NaN,NaN,NaN
1,Lol love the logo! Well done Juli 🙌🙌,2019-09-02,‪+1 (514) 999‑4123‬,NaN,NaN,NaN
2,🙌🏻🙌🏻🙌🏻,2019-09-02,Lucas Afonso,NaN,Brazil,2.0
3,👏🏻👏🏻👏🏻,2019-09-02,Barbara Handam IESE,Brazil,Brazil,4.0
4,B = Bad Bitches,2019-09-02,Chris Coraggio IESE,United States,United States,8.0
5,😂,2019-09-02,Aparna Murali,India,India,7.0
6,Hhahaha loved this 😂,2019-09-02,Julieta Lucci,NaN,Argentina,8.0
7,💪🏼💪🏼💪🏼,2019-09-02,Parsa Ghavimi,NaN,Iran,4.0
8,Oops :),2019-09-02,‪+34 687 11 41 67‬,NaN,NaN,NaN
9,Hahahaha,2019-09-02,‪+1 (514) 999‑4123‬,NaN,NaN,NaN


In [ ]:
emojis_counts.to_csv("051920212300_converted_emoji.csv",index=False)

In [ ]:
def get_most_replied_users(replies_dict):
    most_replied ='|'.join(sorted(replies_dict, key=lambda key: replies_dict[key],reverse=True)[:3])
    return most_replied

In [ ]:
def count_user_replies(df):
    reply_duration_minutes = 2 #minutes
    users_replied_to_user = {}
    #iterate through messages
    for index, row in df.iterrows():
        d = row['datetime'] #current messge datetime
        user = row['Name']
        #if user is not in the dictionary add it
        if not user in users_replied_to_user:
            users_replied_to_user[user] = []
        
        #find the messages within a time window after the current message datetime
        user_replies = df[(df['datetime'] > d) & (df['datetime'] < d+datetime.timedelta(minutes=reply_duration_minutes))]['Name']
        users_replied_to_user[user]+=list(set(user_replies)) #only non-duplicates

    #use Counter to count frequency of replies
    for k,v in users_replied_to_user.items():
        users_replied_to_user[k] = Counter(v)
    
    return users_replied_to_user

In [ ]:
replies =   count_user_replies(df)

In [ ]:
print (replies)


{'🏳️\u200d🌈 Best Section IESE 2021!': Counter(), '\u202a+1\xa0(514)\xa0999‑4123\u202c': Counter({'Aparna Murali': 3, 'Parsa Ghavimi': 3, 'Jaime Llorente IESE': 3, 'Julieta Lucci': 2, '\u202a+1\xa0(514)\xa0999‑4123\u202c': 2, 'Francisco Sá IESE': 2, '\u202a+55\xa021\xa098712‑5391\u202c': 2, 'Jasson': 2, 'Lucas Afonso': 1, 'Chris Coraggio IESE': 1, '\u202a+34\xa0687\xa011\xa041\xa067\u202c': 1, 'Barbara Handam IESE': 1, 'Hari Harsha': 1}), 'Lucas Afonso': Counter({'Lucas Afonso': 276, 'Jasson': 59, 'Francisco Sá IESE': 46, 'Claudia IESE 🇪🇸': 46, 'Gonzalo Diaz': 37, 'Chris Coraggio IESE': 32, 'Parsa Ghavimi': 31, 'Barbara Handam IESE': 29, 'Dan Faermark': 28, 'Andy Klein Iese': 27, 'Thais Cirenza': 24, 'Juana Toranzos': 24, 'Jose IESE': 23, 'Jorge Arbolí': 20, 'Pia Fabbroni': 19, 'Julieta Lucci': 18, 'Jordan Altaras': 17, 'Natalia Krüger IESE': 15, 'Natalia Krüger': 12, 'Roberto Siman': 12, 'Disha IESE': 12, 'Eugenia Delosrios IESE': 12, 'Krishna Chaitanya': 11, 'Dino IESE': 11, 'Matthieu

In [ ]:
replies_reshaped =[]
for k,v in replies.items():
    replies_reshaped.append([k, get_most_replied_users(v)])

replies_reshaped_df = pd.DataFrame(replies_reshaped)

print(replies_reshaped_df)



                               0                                                  1
0   🏳️‍🌈 Best Section IESE 2021!                                                   
1            ‪+1 (514) 999‑4123‬    Aparna Murali|Parsa Ghavimi|Jaime Llorente IESE
2                   Lucas Afonso              Lucas Afonso|Jasson|Francisco Sá IESE
3            Barbara Handam IESE       Barbara Handam IESE|Jasson|Francisco Sá IESE
4            Chris Coraggio IESE            Chris Coraggio IESE|Gonzalo Diaz|Jasson
..                           ...                                                ...
83           ‪+62 813‑7368‑3039‬                 Chris Coraggio IESE|Juana Toranzos
84               Peter Shim IESE              Peter Shim IESE|Jasson|Ana Paula IESE
85               ‪+354 844 4495‬                    Harry HH IESE|Francisco Sá IESE
86            ‪+34 697 86 46 42‬   Andrea Biancardi|‪+34 697 86 46 42‬|Pia Fabbroni
87            ‪+351 913 401 922‬  Eugenia Delosrios IESE|‪+351 913 401 922‬|

In [ ]:
replies_reshaped_df.to_csv("051920212300_replies_df.csv",index=False)